# Ipython Notebook FOR Mass Spectrometry (INFORMS)

In [ ]:
from MS_data_functions import *

If you want plots to be more suitable for presentations (i.e. thicker lines and larger labels), then run the following cell:

In [ ]:
mpl_style('presentation')

If you want plots to appear in a separate window then run the following (you may have to run it twice; inline behaviour can be reset with ```%matplotlib inline```):

In [ ]:
%matplotlib qt

## Load calibration files
There are some default parameters, so this doesn't have to be run, but obviously more up to date calibration files are better.

In [ ]:
I_factors = IonizationFactors(fname='C:/Users/vgx18551/Documents/Data/Metal_oxides/MS_files/calibration/191114_LF_Ifactors.txt')
all_sfs = FragmentationRatios(fname='C:/Users/vgx18551/Documents/Data/Metal_oxides/MS_files/calibration/191114_LF_frag_rats.txt')

## Set up Experimental data
Initialize experiment instance; passing a string to ```Experiment``` will set a default file path—if your log and csv files are in different folders then leave arguments empty and set each full filepath separately when required (e.g. ```data = Experiment()``` followed by ```data.assign_csvfpath("C:/mydata.csv")``` etc.).

In [ ]:
data = Experiment('C:/Users/vgx18551/Documents/Data/Metal_oxides/MS_files/171110_TJW404_HF_R1_blank/',
                  I_factors=I_factors, sfs=all_sfs)

## Mass spectrometry data
### Loading and processing raw .csv files
If this has been done previously and the data saved, then skip to [next section](#Load-previously-fitted-MS-data). Set filepaths/names to MS .csv file and then extract gas fractions (the number in the ```extract_MS_data``` method refers to the number of histograms to sum over:

In [ ]:
data.assign_csv_fpath('171110_TJW404_HF_R1_blank.csv')
data.extract_MS_data(20)

Raw MS data may be plotted as a filled contour plot with the ```zscale``` argument as ```'linear'```, ```'sqrt'``` or ```'log'```; ```t_range``` refers to the time values (in minutes) to plot over:

In [ ]:
fig, ax, cbar = data.MS_contour(zscale='sqrt', t_range=[0, 2000])

Recent calibrations should have been run, but it is always best to self calibrate in so far as it's possible. For example, when running ammonia, then there might well be a period at the end of the run when pure ammonia is used. It would then be appropriate to calibrate the ammonia fragmentation ratios over those times. If a mistake is made then ```all_sfs.reset()``` can be used.

In [ ]:
data.all_sfs.self_calibrate('NH3', data, x_range=[1900, 1950])

Once the MS data are calibrated then they may be fit to actual gas fractions (as opposed to the recorded m/z fractions); a time range (```t_range```) may be used if wanted:

In [ ]:
data.fit_MS_data(t_range=[0, 2000])

Fits to MS data may be plotted using the ```mspec_plot``` method:

In [ ]:
data.plot_MS_fits()

Unfortunately the H<sub>2</sub> ionization factor is notoriously changeable, which means that the hydrogen gas factor probably shouldn't be trusted <i>unless</i> you can change it relative to a known other gas. In the case of ammonia decomposition then the ratio of hydrogen to nitrogen should be 3. Therefore it's good to check what the ratio of the components is:

In [ ]:
data.plot_MS_ratios('h2', 'n2', t_range=[750, 1500])

And then to modify the (hydrogen) ionization ratio and fit the MS data again in order to get a closer average. The ionization ratios can be reset by the code ```data.Is.reset()``` and the actual factor values can be checked by ```print(data.Is)```.

In [ ]:
data.Is.all_Is['h2'] /= 1.3

In [ ]:
data.fit_MS_data(t_range=[0, 2000])

And finally check the ratio again (these last two steps may have to be iterated to get a satisfactory result):

In [ ]:
data.plot_MS_ratios('h2', 'n2', t_range=[750, 1500])

Once the fits are satisfactory then they may be saved as a csv file with the following method (the program will automatically prepend the data.fpath attribute to the filepath argument in the ```save_MS_fits``` method; if you don't want that to happen—e.g. if it's going to a different folder—then run ```data.fpath = ''``` first):

In [ ]:
data.save_MS_fits("fitted_MS_data.csv")

### Load previously fitted MS data
If data is deuterated, then you need to use the ```deuterated=True``` keyword argument, otherwise it defaults to hydrogenated data:

In [ ]:
data.load_MS_fits("fitted_MS_data.csv")

## Load Experiment.vi log files
If the Cambridge Reactor Design reactor has been used, then the keyword argument ```CRD``` needs to be set to ```True``` within the ```get_log_data``` method, otherwise it defaults to the main tube furnace setup:

In [ ]:
data.assign_log_fpath('171110_TJW404_HF_R1_blank.log')
data.get_log_data(CRD=True)

Columns from the log data file can be plotted using the ```plot_log``` method. The acceptable strings can be printed using ```print(data.log_data.columns)```:

In [ ]:
data.plot_log(['MFM'], ylabel='Flow / sccm')

## Looking at log and MS data together
This section assumes that both MS and log data have been loaded [here](#Load-previously-fitted-MS-data) and [here](#Load-Experiment.vi-log-files). The mass spectrometry data can be plotted against temperature using the ```plot_MS_vs_T``` method:

In [ ]:
data.plot_MS_vs_T()

### Calculating and fitting conversion
First of all get the temperature changes:

In [ ]:
data.get_temp_changes()

Where the program calculates the temperature changes can then be plotted:

In [ ]:
data.Tc_indices.plot_changes()

Inspecting the temperature change indices plot above normally shows up some points that should be removed. This can be done using the ```remove``` method as follows (and then replot to check; if you make a mistake then the ```data.Tc_indices.reset()``` method can be used):

In [ ]:
data.Tc_indices.remove([0])
data.Tc_indices.plot_changes()

Once satisfactory temperature changes have been calculated, then the sigmoid points can be calculated and plotted using the following methods:

In [ ]:
data.get_sigmoid_data()

In [ ]:
data.plot_conv_only()

Now it remains to fit the conversion. Using the ```fit_single_Te``` method requires two parameters: (i) $T_{e}$, the temperature (in K) when conversion = $1-e^{-1} \approx 0.632$; (ii) $E_{A}$, which is an activation energy in kJ mol<sup>-1</sup>. Using the ```fit_double_Te``` method requires five parameters: (i) $T_{e1}$; (ii) $E_{A1}$; (iii) $T_{e2}$; (iv) $E_{A2}$; (v) $f$, which is the fraction of sigmoid 1 and 1 - fraction of sigmoid 2.

Guesses can be plotted using the ```plot_conv_guesses``` method:

In [ ]:
data.plot_conv_guesses([800, 180, 650, 80, 0.8])

In [ ]:
data.fit_single_Te([800, 100])

In [ ]:
data.fit_double_Te([800, 180, 650, 80, 0.8])

After fitting, the single fit parameters are stored as ```data.conv_single_params``` and the double sigmoid fit parameters as ```data.conv_double_params```. These latter should be checked to ensure no nonsensical (non-zero) values. Covariances and correlations are also stored as ```data.conv_single_cov``` or ```data.conv_single.corr``` etc.  The fits may be plotted using the ```plot_conv_fit``` function, which will plot both fits by default.

In [ ]:
data.plot_conv_fit()

### Saving conversions and sigmoid fits
The ```save_sigfit``` method will save the temperature in C and the conversion (as opposed to the temperature in K and 1 - fraction of ammonia). The ```fit_type``` argument should be 'single' or 'double' depending on which fit is being saved.

In [ ]:
data.save_sigfit('test.txt')